In [1]:
import sys
sys.path.append('..')

import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
from datetime import datetime
from sqlalchemy import create_engine, text

from utilities_amigocloud import AmigocloudFunctions

In [2]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
ID_PROYECTO = 35248

In [3]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [4]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_amigocloud():
    query = '''SELECT parte.*, plan.unidad_01 
                FROM dataset_367119 parte 
                JOIN dataset_360912 plan 
                ON ST_Intersects(parte.referencia, plan.geometry);'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def obtener_propiedad_plan_amigocloud(cod_prop):
    query = f'''SELECT * FROM dataset_360912 WHERE unidad_01 = {cod_prop}'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def actualizar_lotes_db(unidad_01, unidad_03, ors, row):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    idd= :id,
                    fecha = :fecha,
                    semana = :semana,
                    mezcla = :mezcla,
                    temp_max = :temp_max,
                    temp_min = :temp_min,
                    temp = :temp,
                    viento = :viento,
                    humedad = :humedad,
                    cultivo = :tipo_cultivo,
                    desarrollo = :tipo_desarrollo
                WHERE unidad_01 = :unidad_01
                  AND unidad_03 = :unidad_03
                  AND os = :ors
            """)
            conn.execute(query, {
                "unidad_01": unidad_01,
                "unidad_03": unidad_03,
                "ors": ors,
                "id": row.id,
                "fecha": row.fecha_registro,
                "semana": datetime.fromisoformat(row.fecha_registro).isocalendar().week,
                "mezcla": row.mezcla,
                "temp_max": row.temp_max,
                "temp_min": row.temp_min,
                "temp": (row.temp_max + row.temp_min) / 2,
                "viento": row.viento,
                "humedad": row.humedad,
                "tipo_cultivo": row.tipo_cultivo,
                "tipo_desarrollo": row.tipo_desarrollo
            })
            print(f"✔️ Registros con unidad_01={unidad_01}, unidad_03={unidad_03}, os={ors} marcados como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def copiar_parte_diario_a_clon_amigocloud(id):
    query = f"""INSERT INTO dataset_367584 (id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia) 
                SELECT id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia 
                FROM dataset_367119 
                WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_parte_diario_amigocloud(id):
    query = f"""DELETE FROM dataset_367119 WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def copiar_planificacion_a_clon_amigocloud(unidad_01, ors):
    query = f"""INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry) 
                SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry 
                FROM dataset_360912 
                WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_planificacion_amigocloud(unidad_01, ors):
    query = f"""DELETE FROM dataset_360912 WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def convertir_json_a_geodataframe(json_data, nom_campo_geo):
    # Convertimos a DataFrame para manipulación tabular
    df = pd.DataFrame(json_data)
    # Convertimos la columna geometry (WKB hexadecimal) a objetos shapely
    df[nom_campo_geo] = df[nom_campo_geo].apply(lambda x: wkb.loads(bytes.fromhex(x)))
    # Creamos el GeoDataFrame en CRS original (EPSG:4326)
    gdf = gpd.GeoDataFrame(df, geometry=nom_campo_geo, crs='EPSG:4326')
    # Reproyectamos a UTM Zona 20S (EPSG:32720)
    gdf_utm20s = gdf.to_crs(epsg=32720)
    return gdf_utm20s

In [5]:
parte_diario = obtener_parte_diario_amigocloud()

In [6]:
df_parte_diario = convertir_json_a_geodataframe(parte_diario, 'referencia')

In [7]:
df_parte_diario["mezcla"] = pd.to_numeric(df_parte_diario["mezcla"], errors='coerce').fillna(0).astype(int)

In [8]:
df_parte_diario

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo,tipo,mezcla,unidad_01
0,44,8b14fa0e64e8468797e722966cf17e19,2025-05-14 10:55:39+00:00,POINT (466621.936 8099726.758),27.0,25.0,1.0,83.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,265
1,45,a2008136127a4e128f06aa9eeb1d798b,2025-05-14 10:56:15+00:00,POINT (466796.938 8100089.471),27.0,25.0,1.0,83.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,1816
2,46,4ddef0a645c44880aab71f84f9b1f22c,2025-05-14 10:57:09+00:00,POINT (501711.608 8095097.756),29.0,27.0,3.5,78.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,0,30
3,47,d7160979e3c340a5abf61d6575f49520,2025-05-14 10:58:07+00:00,POINT (468117.068 8077417.566),25.0,23.0,2.0,78.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,2124
4,48,29d367e0986d40188a66c3168cc02e39,2025-05-14 10:58:58+00:00,POINT (429651.918 8089501.993),31.0,29.0,3.0,75.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,1618
5,49,c786c762e73a4df8b2ac7d722e743cbe,2025-05-14 10:59:50+00:00,POINT (478344.525 8089332.517),28.0,26.0,1.0,68.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,1289
6,50,ba78a65bf0944cd28d83dd578b40cad8,2025-05-14 11:00:32+00:00,POINT (468392.752 8101703.658),27.0,25.0,1.0,74.0,None,OTROS,CRECIMIENTO,PARTE,0,2271
7,51,675dd57e6767486d9d95ceff04345ab4,2025-05-14 11:01:16+00:00,POINT (472867.748 8095630.351),27.0,25.0,0.0,78.0,None,CAÑA DE AZUCAR,MADURACION,PARTE,0,17


In [9]:
for i, row in df_parte_diario.iterrows():
    cod_prop = row.unidad_01
    prop = obtener_propiedad_plan_amigocloud(cod_prop)
    gdf_prop = convertir_json_a_geodataframe(prop, 'geometry')
    # se queda con las combinacion unicas de unidad_01, unidad_03 y os
    gdf_prop_unicas = gdf_prop.drop_duplicates(subset=["unidad_01", "unidad_03", "os"])
    for j, row2 in gdf_prop_unicas.iterrows():
        print(row2.os)
        actualizar_lotes_db(row2.unidad_01, row2.unidad_03, row2.os, row)
        copiar_planificacion_a_clon_amigocloud(row2.unidad_01, row2.os)
        eliminar_planificacion_amigocloud(row2.unidad_01, row2.os)
    copiar_parte_diario_a_clon_amigocloud(row.id)
    eliminar_parte_diario_amigocloud(row.id)
    print(f'SE PROCESO PROPIEDAD: {row2.unidad_02}')

41
✔️ Registros con unidad_01=265, unidad_03=2549, os=41 marcados como procesado.
SE PROCESO PROPIEDAD: LOS MAGUESES--BALCAZAR RAQUEL
42
✔️ Registros con unidad_01=1816, unidad_03=1180, os=42 marcados como procesado.
SE PROCESO PROPIEDAD: LOS MAGUESES--BALCAZAR JESUS
37
✔️ Registros con unidad_01=30, unidad_03=41594, os=37 marcados como procesado.
SE PROCESO PROPIEDAD: CAMPO DULCE
39
✔️ Registros con unidad_01=2124, unidad_03=42376, os=39 marcados como procesado.
SE PROCESO PROPIEDAD: TORNO DE LA VIBORA I--HURTADO ANTONIO
40
✔️ Registros con unidad_01=1618, unidad_03=12308, os=40 marcados como procesado.
SE PROCESO PROPIEDAD: MONTE REY I--ROCA MAURICIO
38
✔️ Registros con unidad_01=1289, unidad_03=30035, os=38 marcados como procesado.
SE PROCESO PROPIEDAD: LA PASCANA
43
✔️ Registros con unidad_01=2271, unidad_03=15284, os=43 marcados como procesado.
SE PROCESO PROPIEDAD: SAN MARTIN 27
44
✔️ Registros con unidad_01=17, unidad_03=515, os=44 marcados como procesado.
SE PROCESO PROPIEDAD: 